In [32]:
import torch
from torch import nn
import pickle as pkl
import numpy as np
import MinkowskiEngine as ME

In [33]:
filehandler = open("data.pkl","rb")
dataset = pkl.load(filehandler)
filehandler.close()

In [34]:
torch.cuda.is_available()

True

In [35]:
dev = torch.device("cuda")
dev

device(type='cuda')

In [36]:
class Conv3DFeatureDetector(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        size_1 = 100
        size_2 = 200
        self.conv1 = nn.Sequential(
            ME.MinkowskiConvolution(1, 20, 
                      kernel_size=(kernel_size),
                      dimension=3,
                                   ),
            ME.MinkowskiConvolution(20, size_1, 
                      kernel_size=(kernel_size),
                      dimension=3,
                                   ),
            ME.MinkowskiTanh(),
#             nn.MaxPool3d((2, 2, 2)),
        )
        self.fc1 = ME.MinkowskiLinear(size_1, size_2)
        self.relu = ME.MinkowskiTanh()
        self.fc2 = ME.MinkowskiLinear(size_2, 1)
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = ME.MinkowskiFunctional.sigmoid(out)
        return out
    
# class Conv3DFeatureDetector(nn.Module):
#     def __init__(self, kernel_size):
#         super().__init__()
        
#         size_1 = 100
#         size_2 = 200
#         self.conv1 = nn.Sequential(
#             nn.Conv3d(1, size_1, kernel_size=(kernel_size, kernel_size, kernel_size), padding= (kernel_size - 1)//2),
#             nn.Tanh(),
# #             nn.MaxPool3d((2, 2, 2)),
#         )
#         self.fc1 = nn.Linear(size_1, size_2)
#         self.relu = nn.Tanh()
#         self.fc2 = nn.Linear(size_2, 1)
        
#     def forward(self, x):
#         out = self.conv1(x)
#         out = out.permute(0,2,3,4,1)
#         out = self.fc1(out)
#         out = self.relu(out)
#         out = self.fc2(out)
#         out = torch.sigmoid(out)
#         return out

In [37]:
X = np.vstack([x[None,None,:,:,:] for (x,y) in dataset])
X = X.astype(np.float32)
X = torch.from_numpy(X)

Y = np.vstack([y[None,:,:,:,None] for (x,y) in dataset])
Y = Y.astype(np.float32)
Y = torch.from_numpy(Y)
print(X.shape)
print(Y.shape)

X_train = X[:500]
Y_train = Y[:500]

def get_sparse_tensor_from_numpy(X, d=None):
    idxs = torch.where(X)
    num_nonzero_elements = idxs[0].shape[0]
    num_dimensions = len(idxs)
    coords = torch.vstack(idxs)
    coords = coords.type(torch.int32)
    coords = torch.transpose(coords, 0,1).contiguous()
    feat = torch.ones(coords.shape[0],1)
    if d is None:
        X = ME.SparseTensor(feat, coordinates=coords)
    else:
        X = ME.SparseTensor(feat, coordinates=coords, device=d)
    
    return X


torch.Size([1000, 1, 45, 45, 42])
torch.Size([1000, 45, 45, 42, 1])


In [7]:
X_train.mean()

tensor(0.0107)

In [8]:
# X.cuda()
model = Conv3DFeatureDetector(9)
model.to(dev)



Conv3DFeatureDetector(
  (conv1): Sequential(
    (0): MinkowskiConvolution(in=1, out=20, kernel_size=[9, 9, 9], stride=[1, 1, 1], dilation=[1, 1, 1])
    (1): MinkowskiConvolution(in=20, out=100, kernel_size=[9, 9, 9], stride=[1, 1, 1], dilation=[1, 1, 1])
    (2): MinkowskiTanh()
  )
  (fc1): MinkowskiLinear(in_features=100, out_features=200, bias=True)
  (relu): MinkowskiTanh()
  (fc2): MinkowskiLinear(in_features=200, out_features=1, bias=True)
)

In [9]:
learning_rate = 1.0
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [10]:
X_train[:10,0,:,:,:].shape

torch.Size([10, 29, 29, 28])

In [11]:
batch_size = 10
output = None
batch_y = None
batch_x = None
for epoch in range(1000):
#     permutation = torch.randperm(X_torch.size()[0])
    permutation = torch.arange(X_train.size()[0])
    
    total_loss = 0.0
    for i in range(0,X_train.size()[0], batch_size):
#         print(i)
        optimizer.zero_grad()

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = X_train[indices], Y_train[indices]
        batch_y = batch_y.permute(0,4,1,2,3)
        batch_x = get_sparse_tensor_from_numpy(batch_x[:,0,:,:,:],d=dev)
        
        # in case you wanted a semi-full example
        outputs = model.forward(batch_x)
        
        outputs = outputs.dense(shape=batch_y.shape)[0].cpu()
        
        loss = (outputs - batch_y)**2
        loss[batch_y == 1.0] *= 100.0
        loss = loss.mean()
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    print("total loss: ", total_loss)

/home/nishadg/mcs/InverseGraphics/venv/lib/python3.8/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/nishadg/mcs/InverseGraphics/venv/lib/python3.8/site-packages/MinkowskiEngine/MinkowskiSparseTensor.py:525: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  coords = coords // tensor_stride


total loss:  3.8851121962070465
total loss:  3.7883780635893345
total loss:  3.7587246783077717
total loss:  3.730585116893053
total loss:  3.658859457820654
total loss:  3.57476107403636
total loss:  3.5028116293251514
total loss:  3.456698015332222
total loss:  3.4325627870857716
total loss:  3.4195246137678623
total loss:  3.411555614322424
total loss:  3.4061134792864323
total loss:  3.4020851626992226
total loss:  3.398930359631777
total loss:  3.396358869969845
total loss:  3.394200701266527
total loss:  3.3923488818109035
total loss:  3.390732277184725
total loss:  3.389301221817732
total loss:  3.388019882142544
total loss:  3.3868612870573997
total loss:  3.385805308818817
total loss:  3.384835969656706
total loss:  3.383940849453211
total loss:  3.3831100203096867
total loss:  3.3823353201150894
total loss:  3.381610121577978
total loss:  3.380928948521614
total loss:  3.380286894738674
total loss:  3.3796800449490547


KeyboardInterrupt: 

In [38]:
X_test = X[-50:]
Y_test = Y[-50:]

In [39]:
model_cpu = model.cpu()

In [40]:
pred_Y = model.forward(get_sparse_tensor_from_numpy(X_test[:,0,:,:,:]))

In [41]:
permuted_Y_test = Y_test.permute(0,4,1,2,3)
pred_Y = pred_Y.dense(shape=permuted_Y_test.shape)[0]
pred_Y = pred_Y.permute(0,2,3,4,1)
pred_Y.shape

torch.Size([50, 45, 45, 42, 1])

In [42]:
pred_Y[Y_test[-100:] == 1.0]

tensor([0.0000, 0.0000, 0.0000,  ..., 0.2067, 0.1023, 0.0441],
       grad_fn=<IndexBackward0>)

In [43]:
pred_Y[Y_test[-100:] == 0.0]

tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<IndexBackward0>)

In [44]:
with open("result.pkl","wb") as f:
    pkl.dump((X_test.cpu().detach().numpy(), pred_Y.cpu().detach().numpy()), f)